In [39]:
import pandas as pd
import re


In [23]:
# Import mapstreet data 

mapstreet = pd.read_csv("src_data\GISMAPSTREET.txt", sep="|", header=None, skip_blank_lines=True,
names=["MINORGRIDNAME", "STREETNAME", "ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "ZIPCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME", "MUNICIPALITYNAME", "NUMCENTERLINES", "INTERSECTINGLENGTH", "GEOM_WKT"])

print(mapstreet.columns)

# mapstreet.dropna(inplace=True)
# mapstreet.drop_duplicates(keep='first', inplace=True)
mapstreet.drop(columns=["ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME"], inplace=True)

# dropped_cols = mapstreet.drop(columns=["ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME"])

# mapstreet = mapstreet.drop(columns=["ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME"])
# mapstreet.fillna("", inplace=True)

# print(dropped_cols)
print(mapstreet)
mapstreet.dtypes


Index(['MINORGRIDNAME', 'STREETNAME', 'ROADDIRECTIONPREFIXCD', 'ROADNAME',
       'ROADTYPESUFFIXCD', 'ROADDIRECTIONSUFFIXCD', 'ZIPCD', 'RIGHTZIPCD',
       'LOWADDRESSRANGEVALUE', 'HIGHADDRESSRANGEVALUE', 'TOWNSHIPNAME',
       'MUNICIPALITYNAME', 'NUMCENTERLINES', 'INTERSECTINGLENGTH', 'GEOM_WKT'],
      dtype='object')
        MINORGRIDNAME          STREETNAME      ZIPCD  MUNICIPALITYNAME  \
0       MINORGRIDNAME          STREETNAME  LEFTZIPCD  MUNICIPALITYNAME   
1               AB12B            117TH PL      46394           HAMMOND   
2               AB12B            118TH ST      46394           HAMMOND   
3               AB12B            119TH ST      46394           HAMMOND   
4               AB12B            120TH ST      46394           HAMMOND   
...               ...                 ...        ...               ...   
168648          YN31D   COUNTY ROAD 400 E    46711.0               NaN   
168649          YN31D            E 1175 S    46711.0               NaN   
168650    

C:\Users\Spencer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (6,7,8,9,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


MINORGRIDNAME         object
STREETNAME            object
ZIPCD                 object
MUNICIPALITYNAME      object
NUMCENTERLINES        object
INTERSECTINGLENGTH    object
GEOM_WKT              object
dtype: object

In [21]:
# We can only map lines with WKT coordinates. Drop all rows which have no value in GEO_WKT
mapstreet.dropna(subset=['GEOM_WKT'], how='all', inplace=True)
print(mapstreet)
mapstreet.to_csv(r'./test_data/mapstreet-test.csv', header=True, index=None)

        MINORGRIDNAME          STREETNAME      ZIPCD  MUNICIPALITYNAME  \
0       MINORGRIDNAME          STREETNAME  LEFTZIPCD  MUNICIPALITYNAME   
1               AB12B            117TH PL      46394           HAMMOND   
2               AB12B            118TH ST      46394           HAMMOND   
3               AB12B            119TH ST      46394           HAMMOND   
4               AB12B            120TH ST      46394           HAMMOND   
...               ...                 ...        ...               ...   
168648          YN31D   COUNTY ROAD 400 E    46711.0               NaN   
168649          YN31D            E 1175 S    46711.0               NaN   
168650          YN31D            E 1175 S    46740.0               NaN   
168651          YN32A  COUNTY ROAD 1150 S    46711.0               NaN   
168652          YN32A             S 450 E    46711.0               NaN   

        NUMCENTERLINES  INTERSECTINGLENGTH  \
0       NUMCENTERLINES  INTERSECTINGLENGTH   
1                  

In [24]:
consequence = pd.read_csv("src_data\Street Consequence - Sheet1.csv", sep=",", header=None, skip_blank_lines=True,
names=["STATE", "MUNICIPALITY", "STREETNAME", "MINORGRIDNAME", "CONSEQUENCE"])

consequence.dropna(inplace=True)
print(consequence)
# consequence.dtypes

      STATE  MUNICIPALITY         STREETNAME  MINORGRIDNAME  \
0     STATE  MUNICIPALITY         STREETNAME  MINORGRIDNAME   
1        IN  FRANCESVILLE        N MORTON ST          GJ04D   
2        IN  FRANCESVILLE        N MORTON ST          GJ04D   
5        IN  FRANCESVILLE             ADA ST          GJ04D   
6        IN  FRANCESVILLE             ADA ST          GJ04D   
...     ...           ...                ...            ...   
3253     IN       WINAMAC  COUNTY ROAD 100 S          II13C   
3254     IN       WINAMAC         W HURON ST          II13C   
3255     IN       WINAMAC    W WASHINGTON ST          II13C   
3256     IN  KENDALLVILLE    HIGHPOINTE XING          UD32C   
3259     IN    FORT WAYNE        LANGFORD LN          VI18A   

             CONSEQUENCE  
0     Street Consequence  
1         0.001524869902  
2         0.001524869902  
5         0.001524869902  
6         0.001524869902  
...                  ...  
3253       0.02279010258  
3254       0.02279010258  


In [25]:
merged_df = pd.merge(consequence, mapstreet, on="STREETNAME")

# merged_df.dropna(axis=1, inplace=True)
# merged_df.dropna(axis=0, inplace=True)
# merged_df.drop(columns=["MINORGRIDNAME_y", "ROADDIRECTIONPREFIXCD", "ROADNAME", "ROADTYPESUFFIXCD", "ROADDIRECTIONSUFFIXCD", "LEFTZIPCD", "RIGHTZIPCD", "LOWADDRESSRANGEVALUE", "HIGHADDRESSRANGEVALUE", "TOWNSHIPNAME", "MUNICIPALITYNAME"], inplace=True)
merged_df.rename(columns={"MINORGRIDNAME_x" : "MINORGRIDNAME"}, inplace=True)

print(merged_df)
merged_df.dtypes

        STATE  MUNICIPALITY       STREETNAME  MINORGRIDNAME  \
0       STATE  MUNICIPALITY       STREETNAME  MINORGRIDNAME   
1          IN  FRANCESVILLE      N MORTON ST          GJ04D   
2          IN  FRANCESVILLE      N MORTON ST          GJ04D   
3          IN  FRANCESVILLE      N MORTON ST          GJ04D   
4          IN  FRANCESVILLE      N MORTON ST          GJ04D   
...       ...           ...              ...            ...   
318587     IN  KENDALLVILLE  HIGHPOINTE XING          UD32C   
318588     IN    FORT WAYNE      LANGFORD LN          VI18A   
318589     IN    FORT WAYNE      LANGFORD LN          VI18A   
318590     IN    FORT WAYNE      LANGFORD LN          VI18A   
318591     IN    FORT WAYNE      LANGFORD LN          VI18A   

               CONSEQUENCE MINORGRIDNAME_y      ZIPCD  MUNICIPALITYNAME  \
0       Street Consequence   MINORGRIDNAME  LEFTZIPCD  MUNICIPALITYNAME   
1           0.001524869902           GJ04D      47946               NaN   
2           0.0015

STATE                 object
MUNICIPALITY          object
STREETNAME            object
MINORGRIDNAME         object
CONSEQUENCE           object
MINORGRIDNAME_y       object
ZIPCD                 object
MUNICIPALITYNAME      object
NUMCENTERLINES        object
INTERSECTINGLENGTH    object
GEOM_WKT              object
dtype: object

In [27]:
merged_df.to_csv(r'./test_data/combined_test.csv', header=True, index=None)

In [31]:
merged_small = merged_df.drop(columns=["STATE", "MUNICIPALITY", "MINORGRIDNAME_y", "ZIPCD", "MUNICIPALITYNAME", "NUMCENTERLINES", "INTERSECTINGLENGTH"])
merged_small.drop(index=0, inplace=True)
merged_small.dropna(subset=['GEOM_WKT', "CONSEQUENCE"], how='all', inplace=True)
merged_small.drop_duplicates(inplace=True)

print(merged_small)
merged_small.to_csv(r'./test_data/combined_test.csv', header=True, index=None)

             STREETNAME MINORGRIDNAME     CONSEQUENCE  \
1           N MORTON ST         GJ04D  0.001524869902   
2           N MORTON ST         GJ04D  0.001524869902   
3           N MORTON ST         GJ04D  0.001524869902   
7                ADA ST         GJ04D  0.001524869902   
8                ADA ST         GJ04D  0.001524869902   
...                 ...           ...             ...   
318587  HIGHPOINTE XING         UD32C   0.01842628344   
318588      LANGFORD LN         VI18A   0.01713541548   
318589      LANGFORD LN         VI18A   0.01713541548   
318590      LANGFORD LN         VI18A   0.01713541548   
318591      LANGFORD LN         VI18A   0.01713541548   

                                                 GEOM_WKT  
1       LINESTRING (-86.8771235687437 40.9900292583097...  
2       LINESTRING (-86.8771762640562 40.9886501345356...  
3       LINESTRING (-85.2717313048075 41.4437982175931...  
7       LINESTRING (-86.8783938439566 40.9886549541385...  
8       LINESTR

In [67]:
select_rows = merged_small.tail(n=18)
print(select_rows)

             STREETNAME MINORGRIDNAME    CONSEQUENCE  \
318574  W WASHINGTON ST         II13C  0.02279010258   
318575  W WASHINGTON ST         II13C  0.02279010258   
318576  W WASHINGTON ST         II13C  0.02279010258   
318577  W WASHINGTON ST         II13C  0.02279010258   
318578  W WASHINGTON ST         II13C  0.02279010258   
318579  W WASHINGTON ST         II13C  0.02279010258   
318580  W WASHINGTON ST         II13C  0.02279010258   
318581  W WASHINGTON ST         II13C  0.02279010258   
318582  W WASHINGTON ST         II13C  0.02279010258   
318583  W WASHINGTON ST         II13C  0.02279010258   
318584  W WASHINGTON ST         II13C  0.02279010258   
318585  W WASHINGTON ST         II13C  0.02279010258   
318586  W WASHINGTON ST         II13C  0.02279010258   
318587  HIGHPOINTE XING         UD32C  0.01842628344   
318588      LANGFORD LN         VI18A  0.01713541548   
318589      LANGFORD LN         VI18A  0.01713541548   
318590      LANGFORD LN         VI18A  0.0171354

In [101]:
features_list = []

for index, row in select_rows.iterrows():
    
    # Initialize list of coordinate pairs for the row
    coord_list_of_list = []

    if row['GEOM_WKT'].startswith('LINESTRING'):
        # Get coordinates for a line string
        coordinates_str = re.findall('\(([^)]+)', row['GEOM_WKT'])[0]
        coordinates_list = coordinates_str.split(",")
        
        for coordinate_pair in coordinates_list:
            coord_list_of_list.append(coordinate_pair.split())

            # if coord_list_of_list:
    #     # print('we have coordinates')

    #     for long, lat in coord_list_of_list:
    #         # print(long, ', ', lat)

    #         feature =  { 
    #             "type": "Feature",
    #             "geometry": {
    #                 "type": "LineString",
    #                 "coordinates": [
    #                 [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
    #                 ]
    #                 },
    #             "properties": {
    #                 "prop0": "value0",
    #                 "prop1": 0.0
    #                 }
    #             }

            # for longs,lats in LongLatList
            #     poly = {
            #         'type': 'Polygon',
            #         'coordinates': [[lon,lat] for lon,lat in zip(longs,lats) ]
            #     }
            #     geos.append(poly)

            # geometries = {
            #     'type': 'FeatureCollection',
            #     'features': geos,
            # }

            # geo_str = json.dumps(geometries)

    elif row['GEOM_WKT'].startswith('MULTILINESTRING'):
        # Get coordinates for a multi line string
        # remove leading parenthesis to make this regex happy
        mulitline_coordinates = row['GEOM_WKT'].replace("MULTILINESTRING (", "")
        coordinates_lines = re.findall('\(([^)]+)', mulitline_coordinates)
        # coordinates_str = coordinates_str.replace(",", "")
        # print(coordinates_lines)
        # coordinates_lines_list = coordinates_str.split(",")
        # print(coordinates_lines)
        for line in coordinates_lines:
            # print(line)
            line_split = line.split(",")
            # print(line_split)
            # temporary list to hold list of coordinates for a single line, will be contained in final list of lists
            coord_line_temp = []
            for coordinate_pair in line_split:
                coord_line_temp.append(coordinate_pair.split())
            coord_list_of_list.append(coord_line_temp)
            # print(coord_list_of_list)
            


[[['-85.2243188512971', '40.6556535952111'], ['-85.2229978488102', '40.6556480622007'], ['-85.2222859141948', '40.6556502994865'], ['-85.2213549589778', '40.655653850089']]]
[[['-85.2243188512971', '40.6556535952111'], ['-85.2229978488102', '40.6556480622007'], ['-85.2222859141948', '40.6556502994865'], ['-85.2213549589778', '40.655653850089']], [['-85.2213555853133', '40.6556538213319'], ['-85.2213546298084', '40.6556538494083']]]


In [90]:
test_str = "inside one (inside two), (another two) )"

coordinates_lines = re.findall('\(([^)]+)', test_str)
# coordinates_str = coordinates_str.replace(",", "")
print(coordinates_lines)

['inside two', 'another two']


In [ ]:
    # if coord_list_of_list:
    #     # print('we have coordinates')

    #     for long, lat in coord_list_of_list:
    #         # print(long, ', ', lat)

    #         feature =  { 
    #             "type": "Feature",
    #             "geometry": {
    #                 "type": "LineString",
    #                 "coordinates": [
    #                 [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
    #                 ]
    #                 },
    #             "properties": {
    #                 "prop0": "value0",
    #                 "prop1": 0.0
    #                 }
    #             }

            # for longs,lats in LongLatList
            #     poly = {
            #         'type': 'Polygon',
            #         'coordinates': [[lon,lat] for lon,lat in zip(longs,lats) ]
            #     }
            #     geos.append(poly)

            # geometries = {
            #     'type': 'FeatureCollection',
            #     'features': geos,
            # }

            # geo_str = json.dumps(geometries)